In [1]:
import glob
import os
import imageio


import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.size'] = 12
plt.ioff()

import h5py
import pandas as pd

from skimage.measure import block_reduce

from tqdm.notebook import tqdm



In [2]:
#f = h5py.File("data_files/bnb_WithWire_00.h5")
f = h5py.File("data_files/nue_WithWire_00.h5")


In [3]:
wire_table_rse_num_tups = np.array(f['wire_table']['event_id'], dtype=np.uint32)
particle_table_rse_num_tups = np.array(f['particle_table']['event_id'], dtype=np.uint32)

wire_table_rse_nums = wire_table_rse_num_tups[:, 0] * 1_000_000_000_000 + wire_table_rse_num_tups[:, 1] * 1_000_000 + wire_table_rse_num_tups[:, 2]
particle_table_rse_nums = particle_table_rse_num_tups[:, 0] * 1_000_000_000_000 + particle_table_rse_num_tups[:, 1] * 1_000_000 + particle_table_rse_num_tups[:, 2]

unique_rse_nums = np.unique(wire_table_rse_nums)


In [4]:
all_adcs = f['wire_table']['adc']
all_local_wires = f['wire_table']['local_wire']
all_local_planes = f['wire_table']['local_plane']

particle_start_wire_pos = f['particle_table']['start_wire_pos']
particle_start_wire_time = f['particle_table']['start_wire_time']

os.system("rm event_display_images_hires/*.png")

num_events = 20
for event_i in tqdm(range(num_events)):

    curr_rse_num = unique_rse_nums[event_i]
    curr_run_num = int(curr_rse_num // 1_000_000_000_000)
    curr_subrun_num = int((curr_rse_num % 1_000_000_000_000) // 1_000_000)
    curr_event_num = int(curr_rse_num % 1_000_000)

    # Only looking at first particle to get neutrino vertex info, for some reason the vertex from the event_table tree didn't seem to match.
    event_particle_start_wire_pos = np.array(particle_start_wire_pos[particle_table_rse_nums == unique_rse_nums[event_i]])[0]
    event_particle_start_wire_time = np.array(particle_start_wire_time[particle_table_rse_nums == unique_rse_nums[event_i]])[0]

    event_adcs = np.array(all_adcs[wire_table_rse_nums == unique_rse_nums[event_i]])
    event_local_wire_nums = np.array(all_local_wires[wire_table_rse_nums == unique_rse_nums[event_i]]).flatten()
    event_local_plane_nums = np.array(all_local_planes[wire_table_rse_nums == unique_rse_nums[event_i]]).flatten()

    collection_adcs = event_adcs[event_local_plane_nums == 2]
    collection_local_wire_nums = event_local_wire_nums[event_local_plane_nums == 2]
    collection_particle_start_wire_pos = event_particle_start_wire_pos[2]
    collection_particle_start_wire_time = event_particle_start_wire_time[0]

    wire_pitch = 0.3 # cm, from https://microboone.fnal.gov/wp-content/uploads/MICROBOONE-NOTE-1083-PUB.pdf
    time_tick = 0.5 # microseconds, from https://microboone.fnal.gov/wp-content/uploads/MICROBOONE-NOTE-1083-PUB.pdf
    drift_velocity = 0.114 # cm / microsecond # 114 cm / millisecond, from https://microboone.fnal.gov/wp-content/uploads/MICROBOONE-NOTE-1026-PUB.pdf

    nu_vtx_z = collection_particle_start_wire_pos * wire_pitch # measuring from leftmost wire, not exactly the same as detector coordinates!
    nu_vtx_x = collection_particle_start_wire_time * time_tick * drift_velocity # approximate and relative, not exactly the same as detector coordinates!

    f_downsample = 6
    collection_adcs = block_reduce(collection_adcs, block_size=(1, f_downsample), func=np.sum)

    adccutoff = 10.*f_downsample/6.
    adcsaturation = 100.*f_downsample/6.
    collection_adcs[collection_adcs < adccutoff] = 0
    collection_adcs[collection_adcs > adcsaturation] = adcsaturation

    extents = [0, collection_adcs.shape[0] * wire_pitch, 0, collection_adcs.shape[1] * time_tick * f_downsample * drift_velocity]

    fig = plt.figure(figsize=(1, 1))
    ax = plt.gca()

    # missing background is blue, in case the neutrino is near the edge when we crop it
    plt.imshow(np.zeros((100, 100)), cmap='jet', extent=[-10000, 10000, -10000, 10000])

    plt.imshow(collection_adcs.T, origin='lower', cmap='jet', extent=extents)
    #plt.colorbar()
    #plt.scatter(nu_vtx_z, nu_vtx_x, color='r', s=100, marker='*')
    #plt.xlabel('z (cm)')
    #plt.ylabel('x (cm)')
    #plt.title(f'Event {event_i} Run {curr_run_num} Subrun {curr_subrun_num} Event {curr_event_num}\nCollection Plane')

    plt.axis('off')

    plt.xlim(nu_vtx_z - 100, nu_vtx_z + 100)
    plt.ylim(nu_vtx_x - 100, nu_vtx_x + 100)

    fig.subplots_adjust(bottom = 0)
    fig.subplots_adjust(top = 1)
    fig.subplots_adjust(right = 1)
    fig.subplots_adjust(left = 0)

    plt.savefig(f"event_display_images_hires/event_{event_i}.png", pad_inches=0, dpi=600)
    plt.savefig(f"event_display_images_lores/event_{event_i}.png", pad_inches=0, dpi=128)
    plt.close(fig)


  0%|          | 0/20 [00:00<?, ?it/s]

In [5]:
images = []
for filename in glob.glob("event_display_images_hires/*.png"):
    images.append(imageio.imread(filename))
imageio.mimsave('event_displays_hires.gif', images, loop=0, duration=100)

images = []
for filename in glob.glob("event_display_images_lores/*.png"):
    images.append(imageio.imread(filename))
imageio.mimsave('event_displays_lores.gif', images, loop=0, duration=100)


/tmp/ipykernel_493040/447161657.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))
/tmp/ipykernel_493040/447161657.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))
